In [3]:
from datasets import list_datasets, load_dataset
from pprint import pprint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [1]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [2]:
emoevent_raw = load_dataset('fmplaza/EmoEvent', use_auth_token=True)

No config specified, defaulting to: emo_event/es
Found cached dataset emo_event (/home/juanjo/.cache/huggingface/datasets/fmplaza___emo_event/es/1.1.0/05d5c93b8773e3d90b52effb7132b04cd8b2ccea8d4a0750989ec3aaeb59cad4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
map_dict = {0:"ira", 1:"miedo", 2:"tristeza", 3:"alegria", 4:"asco", 5:"sorpresa", 6:"otra"}
map_dict_inverse = {v: k for k, v in map_dict.items()}

In [138]:
emoevent_df_train = pd.DataFrame(emoevent_raw['train'])
emoevent_df_test = pd.DataFrame(emoevent_raw['test'])
emoevent_df_val = pd.DataFrame(emoevent_raw['validation'])
new_df = pd.concat([emoevent_df_train, emoevent_df_test, emoevent_df_val])
new_df = new_df[new_df['tweet'] != '']
new_df["emocion"] = new_df["label"].map(map_dict)
new_df = new_df[['tweet','emocion']]
new_df.to_csv('../Data/Emoevent/emoevent.csv', index=False)  

In [3]:
new_df = pd.read_csv('../Data/Emoevent/emoevent.csv')

In [23]:
emoevent_df = pd.read_csv('../Data/Emoevent/emoevent.csv')
emoevent_df = emoevent_df.rename(columns={"tweet": "text"})
emoevent_df['label'] = emoevent_df["emocion"].map(map_dict_inverse)
emoevent_df = emoevent_df[['text', 'label']]
emoevent_df.to_csv('../Data/Huggingface/emoevent_hugging.csv', index=False)

In [24]:
emoevent_df = pd.read_csv('../Data/Huggingface/emoevent_hugging.csv')

In [25]:
train, test = train_test_split(emoevent_df, test_size=0.2, random_state=42)
train.to_csv('../Data/Huggingface/emoevent_hugging_train.csv', index=False) 
test.to_csv('../Data/Huggingface/emoevent_hugging_test.csv', index=False)  

In [6]:
emocions_size = {}
for i in map_dict.values():
    emocions_size[i] = len(new_df[new_df['emocion'] == i])
emocions_size

{'ira': 840,
 'miedo': 95,
 'tristeza': 992,
 'alegria': 1756,
 'asco': 160,
 'sorpresa': 340,
 'otra': 4010}

In [45]:
semval_raw = load_dataset('sem_eval_2018_task_1', 'subtask5.spanish')

Found cached dataset sem_eval_2018_task_1 (/home/juanjo/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.spanish/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
map_dict_semval = {'anger':"ira", 'fear':"miedo", 'sadness':"tristeza", 'joy':"alegria", 'disgust':"asco", 'surprise':"sorpresa", 'other':"otra"}

In [121]:
semval_df_train = pd.DataFrame(semval_raw['train'])
semval_df_test = pd.DataFrame(semval_raw['test'])
semval_df_val = pd.DataFrame(semval_raw['validation'])
semval_new_df = pd.concat([semval_df_train, semval_df_test, semval_df_val])
#semval_new_df = semval_new_df[new_df['tweet'] != '']
#new_df["emocion"] = new_df["label"].map(map_dict)

In [122]:
l = list(semval_new_df.columns)
l2 = ['anticipation', 'optimism', 'pessimism', 'trust', 'love']
l3 = [i for i in l if i not in l2]

In [124]:
semval_new_df_filt = semval_new_df[l3]
semval_new_df_filt['emotiom_raw'] = semval_new_df_filt.iloc[:, 2:].dot(semval_new_df_filt.add_suffix("~~").columns[2:]).str[:-2]
semval_new_df_filt['emotiom_raw'][semval_new_df_filt['emotiom_raw'] == ''] = 'other'
semval_clean = semval_new_df_filt[~semval_new_df_filt['emotiom_raw'].str.contains('~')]

/tmp/ipykernel_13734/1128644199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  semval_new_df_filt['emotiom_raw'] = semval_new_df_filt.iloc[:, 2:].dot(semval_new_df_filt.add_suffix("~~").columns[2:]).str[:-2]
/tmp/ipykernel_13734/1128644199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  semval_new_df_filt['emotiom_raw'][semval_new_df_filt['emotiom_raw'] == ''] = 'other'
/home/juanjo/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats

In [126]:
semval_clean["emocion"] = semval_clean["emotiom_raw"].map(map_dict_semval)
semval_clean = semval_clean.rename(columns={"Tweet": "tweet"})
semval_clean = semval_clean[['tweet','emocion']]
semval_clean.to_csv('../Data/Semval/semval.csv', index=False) 

/tmp/ipykernel_13734/755590043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  semval_clean["emocion"] = semval_clean["emotiom_raw"].map(map_dict_semval)


In [24]:
semval_clean['label'] = semval_clean["emocion"].map(map_dict_inverse)
semval_clean = semval_clean.rename(columns={"tweet": "text"})
semval_clean.to_csv('../Data/Semval/semval.csv', index=False) 

In [18]:
semval_clean = pd.read_csv('../Data/Semval/semval.csv')

In [20]:
train_semval, test_semval = train_test_split(semval_clean[['text', 'label']], test_size=0.2, random_state=42)
train_semval.to_csv('../Data/Huggingface/semval_hugging_train.csv', index=False) 
test_semval.to_csv('../Data/Huggingface/semval_hugging_test.csv', index=False)  

In [11]:
emocions_size_semval = {}
for i in map_dict_semval.values():
    emocions_size_semval[i] = len(semval_clean[semval_clean['emocion'] == i])
emocions_size_semval

{'ira': 914,
 'miedo': 396,
 'tristeza': 849,
 'alegria': 1741,
 'asco': 168,
 'sorpresa': 138,
 'otra': 1016}

In [16]:
sidorov = pd.read_csv('../Data/Sidorov/corpus_marcado6emociones_extracted.csv')

In [38]:
sidorov['emocion'] = sidorov['emocion'].str.lower()
sidorov = sidorov.rename(columns={"text": "tweet"})
sidorov.to_csv('../Data/Sidorov/sidorov.csv', index=False) 

In [40]:
emocions_size_sidorov = {}
for i in map_dict_semval.values():
    emocions_size_sidorov[i] = len(sidorov[sidorov['emocion'] == i])
emocions_size_sidorov

{'ira': 1830,
 'miedo': 1873,
 'tristeza': 282,
 'alegria': 1952,
 'asco': 1862,
 'sorpresa': 1976,
 'otra': 0}

In [45]:
dataset_counts = pd.DataFrame([emocions_size,emocions_size_semval, emocions_size_sidorov], index = ['EmoEvent','SemEval','Sidorov'])

In [48]:
dataset_counts.to_latex()

\begin{tabular}{lrrrrrrr}
\toprule
{} &   ira &  miedo &  tristeza &  alegria &  asco &  sorpresa &  otra \\
\midrule
EmoEvent &   840 &     95 &       992 &     1756 &   160 &       340 &  4010 \\
SemEval  &   914 &    396 &       849 &     1741 &   168 &       138 &  1016 \\
Sidorov  &  1830 &   1873 &       282 &     1952 &  1862 &      1976 &     0 \\
\bottomrule
\end{tabular}



In [27]:
emoevent_hugging_train = pd.read_csv('../Data/Huggingface/emoevent_hugging_train.csv')
semval_hugging_train = pd.read_csv('../Data/Huggingface/semval_hugging_train.csv')
semval_emoevent_hugging_train = pd.concat([emoevent_hugging_train, semval_hugging_train])
semval_emoevent_hugging_train .to_csv('../Data/Huggingface/semval_emoevent_hugging_train.csv', index=False) 
emoevent_hugging_test = pd.read_csv('../Data/Huggingface/emoevent_hugging_test.csv')
semval_hugging_test = pd.read_csv('../Data/Huggingface/semval_hugging_test.csv')
semval_emoevent_hugging_test = pd.concat([emoevent_hugging_train, semval_hugging_test])
semval_emoevent_hugging_test .to_csv('../Data/Huggingface/semval_emoevent_hugging_test.csv', index=False) 

In [13]:
semval_emoevent_hugging_train = pd.concat([emoevent_hugging_train, semval_hugging_train])

In [17]:
semval_emoevent_hugging_test

,text,label
0,"Ante estos resultados, expectativa de avances ...",3
1,"Pobre #Venezuela, tan lejos de Dios y tan cerc...",2
2,¿ARENGAS DE UN GOLPISTA?: Disturbios en Caraca...,6
3,La negociación y el diálogo será la clave para...,6
4,Cersei se encuentra buscando lentillas de colo...,6
...,...,...
1040,@user no se puede jugar desde él sabado ..,2
1041,"¿Por qué gran mierda borras los tuits, @user C...",0
1042,Ayer antes de irme me salió lo de la escoba y ...,0
1043,@user @user Te pasas jejeje,3
